# **Install and Import Necessary Libraries**

installing `translate` for translating the text data to English

In [ ]:
!pip install translate

In [ ]:
# basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from time import time

# for encoding categorical data
from sklearn.preprocessing import LabelEncoder

# for spliting the dataset and determining the best params
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_predict, GridSearchCV

# text processing libraries
from translate import Translator
from string import punctuation
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# sparse matrix
from scipy.sparse import vstack , hstack, csr_matrix

# machine learning algorithm
from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
from sklearn.svm import SVR

# for seeing model performance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# ignore warning
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# **Load & Overview The Dataset**

In [ ]:
# loading the dataset
df_train = pd.read_csv("https://drive.google.com/uc?id=18jWP3_hB7OkTCOZDPEFt8A3EUCNgJmns", sep="|")
df_predict = pd.read_csv("https://drive.google.com/uc?id=1kc2-a6QwcKf2Cte9ISrQ7MojxBN-N7ER", sep="|")

In [ ]:
# overview the top two rows of the dataset
df_train.head(2)

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN


In [ ]:
df_predict.head(2)

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description
0,31747,Sous Chef,Lombok,IDR,Pegawai (non-manajemen & non-supervisor),8 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran",NaN,NaN,NaN,Makanan & Minuman/Katering/Restoran,Candidate must posses at least bachelor degree...
1,31748,"Bancassurance Officer (Area: Bali, Sulawesi Ut...",Palu,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Penjualan / Pemasaran,Penjualan - Jasa Keuangan","Asuransi kesehatan;Waktu regular, Senin - Juma...",14 days,201 - 500 pekerja,Asuransi,Bancassurance Officer adalah Representative da...


In [ ]:
# see the unique currencies on the dataset
print(df_train["salary_currency"].unique())
print(df_predict["salary_currency"].unique())

['IDR' 'USD' nan]
['IDR']


In [ ]:
# drop unnecessary rows on `df_train`
df_train = df_train[df_train['salary'].notna()]
df_train = df_train[df_train['salary_currency'] == 'IDR']

In [ ]:
# pre-filling the `salary` column of `df_predict`
df_predict['salary'] = 0

In [ ]:
# creating combined dataset just in case if it's needed
df_combined = pd.concat([df_train, df_predict]).reset_index()

In [ ]:
# see the shape of the dataset
print(df_train.shape)
print(df_predict.shape)
print(df_combined.shape)

(6350, 15)
(3000, 15)
(9350, 16)


from above result, we need to keep in mind that **index 0 until 6349 are the train set**, whereas **index 6350 until 9349 are the predict set**. This is because we'll be using the `df_combined` for the processing step

In [ ]:
# see the percentage of missing data of each column on the df_combined
percent_miss = pd.DataFrame(
                            {'total_miss': df_combined.isnull().sum(), 
                            'perc_miss': (df_combined.isnull().sum() / len(df_combined.index)) * 100}
                            ) 
percent_miss

,total_miss,perc_miss
index,0,0.000000
id,0,0.000000
job_title,0,0.000000
location,0,0.000000
salary_currency,0,0.000000
career_level,0,0.000000
experience_level,782,8.363636
education_level,0,0.000000
employment_type,0,0.000000
job_function,0,0.000000


# **Processing The Dataset**

cleaning the `experience_level` column values and filling the null values with mean value grouped by `career_level`

In [ ]:
# get the first index of splitted `experience_level` values
df_combined['experience_level'] = df_combined['experience_level'].str.split(" ").str[0]

In [ ]:
df_combined['experience_level'].unique()

array([nan, '3', '1', '2', '5', '4', '10', '6', '8', '7', '15', '12',
       '11', '17', '20', 'Lebih'], dtype=object)

In [ ]:
# excluding 'Lebih' values
df_combined['experience_level'] = df_combined['experience_level'][df_combined['experience_level'] != 'Lebih']

In [ ]:
# changing the data type of `experience_level` to float
df_combined['experience_level'] = df_combined['experience_level'].astype(float)

In [ ]:
# filling the `experience_level` null values by its mean value grouped by `career_level`
df_combined['experience_level'] = df_combined.groupby('career_level')['experience_level'].transform(lambda x: x.fillna(x.mean()))

filling nan values of `job_benefits` column with "None"

In [ ]:
df_combined['job_benefits'] = df_combined['job_benefits'].fillna('None')

one-hot encoding for `job_benefits` column

In [ ]:
# storing all possible categories for `job_benefits`
job_benefit_cols = list(df_combined['job_benefits'].str.get_dummies(sep=';'))

In [ ]:
# applying one-hot encoding for `job_benefits` column
df_ohe_job_benefits = df_combined['job_benefits'].str.get_dummies(';').reindex(job_benefit_cols, fill_value=0, axis=1)
df_combined = df_combined.join(df_ohe_job_benefits)
df_combined.drop('job_benefits', axis=1, inplace=True)

In [ ]:
df_combined.shape

(9350, 1219)

cleaning on `company_process_time`, `company_size`, and `company_industry` columns

In [ ]:
# deleting ' days' string values on `company_process_time` column by splitting the string and taking the first index, 
# which is the number that we're looking for

df_combined['company_process_time'] = df_combined['company_process_time'].str.split(' ').str[0]

# changing its column data type to float
df_combined['company_process_time'] = df_combined['company_process_time'].astype(float)

# filling its column null values by its column mean
df_combined['company_process_time'] = df_combined['company_process_time'].fillna(df_combined['company_process_time'].mean())

In [ ]:
# filling `company_size` null values with 'None'
df_combined['company_size'] = df_combined['company_size'].fillna('None')

In [ ]:
# filling `company_industry` null values with 'None'
df_combined['company_industry'] = df_combined['company_industry'].fillna('None')

In [ ]:
# seeing again the percentage of missing data of each column on the df_combined to make sure that no more null values here
percent_miss = pd.DataFrame(
                            {'total_miss': df_combined.isnull().sum(), 
                            'perc_miss': (df_combined.isnull().sum() / len(df_combined.index)) * 100}
                            ) 
percent_miss

,total_miss,perc_miss
index,0,0.0
id,0,0.0
job_title,0,0.0
location,0,0.0
salary_currency,0,0.0
...,...,...
"tunjangan kerajinan, tunjangan makan, upah lembur",0,0.0
uang makan dan transport,0,0.0
uniform,0,0.0
•\tDaily Allowance,0,0.0


label encoding on `location` column

In [ ]:
# creating LabelEncoder instance
label_encoder = LabelEncoder()

# applying `label_encoder` to the desired column
df_combined['location'] = label_encoder.fit_transform(df_combined['location'])

ordinal label encoding on `career_level` column

In [ ]:
# creating empty list to store ordinal label values
ordinal = []

# iterating each record on `career_level` column to determine which label to be assigned to each level
for x in df_combined['career_level'] :
  if x == "Lulusan baru/Pengalaman kerja kurang dari 1 tahun" :
    ordinal.append(1)
  elif x == "Pegawai (non-manajemen & non-supervisor)" :
    ordinal.append(2)
  elif x == "Supervisor/Koordinator" :
    ordinal.append(3)
  elif x == "Manajer/Asisten Manajer" :
    ordinal.append(4)
  elif x == "CEO/GM/Direktur/Manajer Senior" :
    ordinal.append(5)

# replacing the original values to new value that has been stored on the created list
df_combined['career_level'] = np.array(ordinal)

one-hot encoding on `education_level`, `job_function`, `employment_type`, and `company_industry` columns

In [ ]:
# splitting string value that separated by specific characters
df_combined.education_level = df_combined.education_level.str.split(', ')
df_combined.job_function = df_combined.job_function.str.split(',')

In [ ]:
# one-hot encoding on `education_level` column
df_edu_lv = pd.concat(
                        [
                          df_combined.explode('education_level')
                          .pivot_table(index='id', columns='education_level', aggfunc='size', fill_value=0)
                          .add_prefix('edu_lv_'),
                          df_combined.set_index('id').salary
                        ], axis=1
                    )
df_edu_lv = df_edu_lv.drop(['salary'], axis=1)
df_edu_lv['id_edu_lv'] = df_edu_lv.index
df_combined = df_combined.drop(['education_level'], axis=1)

In [ ]:
# one-hot encoding on `education_level` column
df_job_function = pd.concat(
                              [
                                df_combined.explode('job_function')
                                .pivot_table(index='id', columns='job_function', aggfunc='size', fill_value=0)
                                .add_prefix('job_function_'),
                                df_combined.set_index('id').salary
                              ], axis=1
                          )
df_job_function = df_job_function.drop(['salary'],axis=1)
df_job_function['id_job_function'] = df_job_function.index
df_combined = df_combined.drop(['job_function'], axis=1)

In [ ]:
# one-hot encoding on `employment_type` column
df_employment_type = pd.get_dummies(df_combined.employment_type, prefix='employment_type')
df_employment_type['id_employ_type'] = df_edu_lv.index
df_combined = df_combined.drop(["employment_type"], axis=1)

# one-hot encoding on `company_industry` column
df_company_industry = pd.get_dummies(df_combined.company_industry, prefix='company_industry')
df_company_industry['id_company_industry'] = df_edu_lv.index
df_combined = df_combined.drop(['company_industry'],axis=1)

In [ ]:
df_combined = pd.merge(df_combined, df_edu_lv, left_on='id', right_on='id_edu_lv', how='left').drop('id_edu_lv', axis=1)
df_combined = pd.merge(df_combined, df_job_function, left_on='id', right_on='id_job_function', how='left').drop('id_job_function', axis=1)
df_combined = pd.merge(df_combined, df_employment_type, left_on='id', right_on='id_employ_type', how='left').drop('id_employ_type', axis=1)
df_combined = pd.merge(df_combined, df_company_industry, left_on='id', right_on='id_company_industry', how='left').drop('id_company_industry', axis=1)

ordinal label encoding on `company_size` column

In [ ]:
# creating empty list to store ordinal label values
ordinal = []

# iterating each record on `company_size` column to determine which label to be assigned to each level
for x in df_combined['company_size'] :
  if x == "None" :
    ordinal.append(0)
  elif x == "1- 50 pekerja" :
    ordinal.append(1)
  elif x == "51 - 200 pekerja" :
    ordinal.append(2)
  elif x == "201 - 500 pekerja" :
    ordinal.append(3)
  elif x == "501 - 1000 pekerja" :
    ordinal.append(4)
  elif x == "1001 - 2000 pekerja" :
    ordinal.append(5)
  elif x == "2001 - 5000 pekerja" :
    ordinal.append(6)
  elif x == "Lebih dari 5000 pekerja" :
    ordinal.append(7)

df_combined['company_size'] = np.array(ordinal)

In [ ]:
df_combined.drop(['index', 'salary_currency'], axis=1, inplace=True)

# **Data Splitting**

In [ ]:
# remember that we've information that index 0 until 6349 are the train set, whereas index 6350 until 9349 are the predict set

# selecting the right index for each set
train = df_combined[df_combined.index < 6350]
train = train.drop(['id'], axis=1)
predict = df_combined[df_combined.index > 6349]

In [ ]:
# form the `X`, `y`, `X_predict`, and `predict_id`
X = train.drop(columns='salary')
y = train['salary']

predict_id = np.array(predict['id'])
X_predict = predict.drop(['id', 'salary'], axis=1)

In [ ]:
X.shape

(6350, 1370)

In [ ]:
X_predict.shape

(3000, 1370)

# **Applying NLP on `job_title` and `job_description` columns**

## **Creating Functions**

### **1. Punctuation Removal**

In [ ]:
# Create a list of punctuation replacements
punctuation_symbols = []

for symbol in punctuation:
  punctuation_symbols.append((symbol, ''))

def remove_punctuation(sentence: str) -> str:
  return sentence.translate(str.maketrans('', '', string.punctuation))

### **2. Digits Removal**

In [ ]:
def remove_digits(x):
  x = ''.join([i for i in x if not i.isdigit()])
  return x

### **3. Stop Words Removal**

In [ ]:
stop = stopwords.words('english')

def remove_stopwords(x):
  x = ' '.join([i for i in x.lower().split(' ') if i not in stop])
  return x

### **4. Lowercase Formatting**

In [ ]:
def to_lower(x):
  return x.lower()

### **5. Word Lemmatizer**

In [ ]:
lematizer = WordNetLemmatizer()

# Defining lemmatizer function
def word_lemmatizer(text):
    lemma_text = [lematizer.lemmatize(word) for word in text]
    return lemma_text

### **6. Translator**

In [ ]:
def translator(text):
  text = Translator(to_lang="en").translate(text)
  return text

## **Processing on `job_title` column**

In [ ]:
X['job_title'] = X['job_title'].astype(str)
X_predict['job_title'] = X_predict['job_title'].astype(str)

X['job_title'] = X['job_title'].apply(remove_digits)
X['job_title'] = X['job_title'].apply(remove_punctuation)
X['job_title'] = X['job_title'].apply(remove_stopwords)
X['job_title'] = X['job_title'].apply(to_lower)

X_predict['job_title'] = X_predict['job_title'].apply(remove_digits)
X_predict['job_title'] = X_predict['job_title'].apply(remove_punctuation)
X_predict['job_title'] = X_predict['job_title'].apply(remove_stopwords)
X_predict['job_title'] = X_predict['job_title'].apply(to_lower)

In [ ]:
# Apply Word Lemmatizer to `job_title` column 
X['job_title'] = X['job_title'].apply(lambda x: word_lemmatizer(x))
X_predict['job_title'] = X_predict['job_title'].apply(lambda x: word_lemmatizer(x))

In [ ]:
# Apply Count Vectorizer to `job_title`, this converts it into a sparse matrix 
count_vec = CountVectorizer()

X['job_title'] = X['job_title'].astype(str)
X_predict['job_title'] = X_predict['job_title'].astype(str)

X_job_title = count_vec.fit_transform(X['job_title'])
X_predict_job_title = count_vec.fit_transform(X_predict['job_title'])

## **Processing on `job_description` column**

In [ ]:
X.job_description = X.job_description.astype(str)
X_predict.job_description = X_predict.job_description.astype(str)

X['job_description'] = X['job_description'].map(lambda x: translator(x))
X_predict['job_description'] = X_predict['job_description'].map(lambda x: translator(x))

X['job_description'] = X['job_description'].apply(remove_digits)
X['job_description'] = X['job_description'].apply(remove_punctuation)
X['job_description'] = X['job_description'].apply(remove_stopwords)
X['job_description'] = X['job_description'].apply(to_lower)

X_predict['job_description'] = X_predict['job_description'].apply(remove_digits)
X_predict['job_description'] = X_predict['job_description'].apply(remove_punctuation)
X_predict['job_description'] = X_predict['job_description'].apply(remove_stopwords)
X_predict['job_description'] = X_predict['job_description'].apply(to_lower)

In [ ]:
# Apply Word Lemmatizer to `job_description` column 
X['job_description'] = X['job_description'].apply(lambda x: word_lemmatizer(x))
X_predict['job_description'] = X_predict['job_description'].apply(lambda x: word_lemmatizer(x))

In [ ]:
# Apply TFIDF to `job_description` 
tfidf_vec = TfidfVectorizer()

X['job_description'] = X['job_description'].astype(str)
X_predict['job_description'] = X_predict['job_description'].astype(str)

X_job_description = tfidf_vec.fit_transform(X['job_description'])
X_predict_job_description = tfidf_vec.fit_transform(X_predict['job_description'])

# **Create CSR_Matrix & Merge The Sparse Matrices**

In [ ]:
col_names = df_combined.columns.values.tolist()

In [ ]:
col_names.remove('job_title')
col_names.remove('job_description')
col_names.remove('id')
col_names.remove('salary')

creating Sparse Matrix for `X`

In [ ]:
# Create our final sparse matrix
X_dummies = csr_matrix(pd.get_dummies(X[col_names], sparse=True).values)
# Combine everything together
X_sparse_merge = hstack((X_dummies, X_job_title, X_job_description)).tocsr()

creating Sparse Matrix for `X_predict`

In [ ]:
# Create our final sparse matrix
X_predict_dummies = csr_matrix(pd.get_dummies(X[col_names], sparse=True).values)
# Combine everything together
X_predict_sparse_merge = hstack((X_predict_dummies, X_job_title, X_job_description)).tocsr()

# **Model Training**

## **Searching The Best Params**

In [ ]:
def hyperparams_tuning(X_train, y_train):
    params = {
        'tree_method': ['gpu_hist'],
        'learning_rate': [.03, 0.05, .07],
        'max_depth': [5, 6, 7],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = params,
                           scoring = 'neg_mean_squared_error', # MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [ ]:
hyperparams_tuning(X_sparse_merge, y)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


## **Cross Validation**

In [ ]:
xgb_model = XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.01,
        max_depth = 10,
        min_child_weight = 3,
        n_estimators = 500,
        subsample = 0.7)

scoring = {'MSE': make_scorer(mean_squared_error),
          'MAE': make_scorer(mean_absolute_error)}

kfold = KFold(n_splits=5, random_state=1234, shuffle = True)

results_reg = cross_validate(estimator=xgb_model, 
                             X=X, 
                             y=y, 
                             cv=kfold, 
                             scoring=scoring)

In [ ]:
b_lin = np.sqrt(results_reg.get('test_MSE'))
print('mean_RMSE : ', "%.15f" % (b_lin.mean()))

# **Model Prediction**

In [ ]:
predict_id = np.array(predict['id'])
X_predict = predict.drop(['id', 'salary'], axis=1)

In [ ]:
xgb_model.fit(X_sparse_merge, y)
y_pred = xgb_model.predict(X_predict_sparse_merge)

In [ ]:
submission = pd.DataFrame({"id": pred_id, "salary": y_pred}).reset_index()
submission.to_csv("submission.csv", index=False)